In [1]:
from math import cos, asin, sqrt

# train 
detail_train = pd.read_csv('../coupon_data_project2/coupon_detail_train_translated_en.csv', parse_dates=['I_DATE'])
visit_train = pd.read_csv('../coupon_data_project2/coupon_visit_train.csv', parse_dates=['I_DATE'])

area_train = pd.read_csv('../coupon_data_project2/coupon_area_train_translated_en.csv')
coupon_list_train = pd.read_csv('../coupon_data_project2/coupon_list_train_translated_en.csv', parse_dates=['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND'])

# base data
prefecture_location = pd.read_csv('../coupon_data_project2/prefecture_locations_translated_en.csv')
user_list = pd.read_csv('../coupon_data_project2/user_list_translated_en.csv', parse_dates=['WITHDRAW_DATE', 'REG_DATE'])

# test data
area_test = pd.read_csv('../coupon_data_project2/coupon_area_test_translated_en.csv')
coupon_list_test = pd.read_csv('../coupon_data_project2/coupon_list_test_translated_en.csv', parse_dates=['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND'])

# submisiion
submission = pd.read_csv('../coupon_data_project2/sample_submission.csv')

detail_train['PURCHASE_FLG'] = 1
detail_train.rename(columns = {'I_DATE': 'purchase_date', 'SMALL_AREA_NAME': 'resid_small' }, inplace=True) # 거주지 구분을 위해 컬럼명 변경
visit_train.rename(columns = {'I_DATE': 'view_date', 'VIEW_COUPON_ID_hash':'COUPON_ID_hash'}, inplace=True)

# date의 시간을 제거
detail_train['purchase_date'] = pd.to_datetime(detail_train['purchase_date'].dt.date)
visit_train['view_date'] = pd.to_datetime(visit_train['view_date'].dt.date)

visit_train.drop(labels=['REFERRER_hash', 'SESSION_ID_hash', 'PURCHASEID_hash'], axis=1, inplace=True)
detail_train.drop(labels=['PURCHASEID_hash'], axis=1, inplace=True)


train = pd.merge(detail_train, visit_train, how='outer')

print(detail_train.shape)
print(visit_train.shape)
print(train.shape)

(168996, 6)
(2833180, 5)
(2913043, 8)


In [2]:
area_train.columns

Index(['SMALL_AREA_NAME', 'PREF_NAME', 'COUPON_ID_hash'], dtype='object')

In [3]:
train2 = pd.merge(train, user_list, how='left', on='USER_ID_hash')

In [4]:
train2 = pd.merge(train2, coupon_list_train, how='left', on='COUPON_ID_hash')

In [6]:
train2.rename(columns={'PREF_NAME':'user_pref', 'ken_name':'spot_pref'},inplace=True)

In [8]:
df = train2[['COUPON_ID_hash','USER_ID_hash','user_pref','spot_pref']]

In [9]:
area_train.columns

Index(['SMALL_AREA_NAME', 'PREF_NAME', 'COUPON_ID_hash'], dtype='object')

In [10]:
lookup_table = area_train.groupby('COUPON_ID_hash').apply(lambda x: list(x.PREF_NAME))
lookup_table = pd.DataFrame(lookup_table, columns = ['coupon_pref']).reset_index()

In [40]:
df_loc = pd.merge(df, lookup_table, how='left', on='COUPON_ID_hash')

In [41]:
df_loc['user_pref'] = df_loc['user_pref'].fillna("NA")

In [42]:
df_loc['coupon_pref'] = df_loc['coupon_pref'].fillna("NA_pref")

In [43]:
# user_pref가 coupon_pref에 포함하는지 알아보기
def aa(x, y):
    if x in y:
        return 1
    else:
        return 0

In [44]:
df_loc['PREF_in'] = df_loc.apply(lambda df2: aa(df2['user_pref'], df2['coupon_pref']), axis=1)

In [49]:
prefecture_location = pd.read_csv('../coupon_data_project2/prefecture_locations_translated_en.csv')

In [50]:
prefecture_location.drop('PREFECTUAL_OFFICE', axis=1, inplace=True)

In [51]:
# user_pref의 위도 경도 가져오기
prefecture_location.rename(columns={'PREF_NAME':'user_pref'},inplace=True)
df_loc = pd.merge(df_loc, prefecture_location, how='left', on='user_pref')

# spot_pref의 위도 경도 가져오기
prefecture_location.rename(columns={'user_pref':'spot_pref'},inplace=True)
df_loc = pd.merge(df_loc, prefecture_location, how='left', on='spot_pref')

In [52]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [53]:
df_loc['distance'] = df_loc.apply(lambda df_loc : distance(df_loc['LATITUDE_x'] , df_loc['LONGITUDE_x'] ,df_loc['LATITUDE_y'], df_loc['LONGITUDE_y']), axis=1)

In [54]:
df_csv = df_loc[['COUPON_ID_hash', 'USER_ID_hash', 'distance', 'PREF_in']]
df_csv.to_csv('train_location.csv', encoding='utf-8')

# 테스트 데이터

In [64]:
name_df_coupon_list_test = pd.DataFrame(coupon_list_test['COUPON_ID_hash'])
name_df_user_list = pd.DataFrame(user_list['USER_ID_hash'])

name_df_coupon_list_test['key'] = 0
name_df_user_list['key'] = 0

name_df_test = pd.merge(name_df_coupon_list_test, name_df_user_list, how='outer', on='key')

name_df_test.drop('key', 1, inplace=True)

In [84]:
loc_test = pd.merge(name_df_test, user_list, how='left', on='USER_ID_hash')

loc_test = pd.merge(loc_test, coupon_list_test, how='left', on='COUPON_ID_hash')

In [85]:
loc_test.columns

Index(['COUPON_ID_hash', 'USER_ID_hash', 'REG_DATE', 'SEX_ID', 'AGE',
       'WITHDRAW_DATE', 'PREF_NAME', 'CAPSULE_TEXT', 'GENRE_NAME',
       'PRICE_RATE', 'CATALOG_PRICE', 'DISCOUNT_PRICE', 'DISPFROM', 'DISPEND',
       'DISPPERIOD', 'VALIDFROM', 'VALIDEND', 'VALIDPERIOD', 'USABLE_DATE_MON',
       'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
       'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
       'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'LARGE_AREA_NAME',
       'ken_name', 'SMALL_AREA_NAME'],
      dtype='object')

In [87]:
loc_test.rename(columns={'PREF_NAME':'user_pref', 'ken_name':'spot_pref'},inplace=True)

df_test = loc_test[['COUPON_ID_hash','USER_ID_hash','user_pref','spot_pref']]

In [88]:
# submission[‘PURCHASED_COUPONS’] = submission[‘PURCHASED_COUPONS’].astype('str')
df_test['spot_pref'] = df_test['spot_pref'].apply(lambda x: x.replace(' prefecture',''))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [89]:
lookup_table = area_test.groupby('COUPON_ID_hash').apply(lambda x: list(x.PREF_NAME))
lookup_table = pd.DataFrame(lookup_table, columns = ['coupon_pref']).reset_index()

df_test = pd.merge(df_test, lookup_table, how='left', on='COUPON_ID_hash')

df_test['user_pref'] = df_test['user_pref'].fillna("NA")
df_test['coupon_pref'] = df_test['coupon_pref'].fillna("NA_pref")

# user_pref가 coupon_pref에 포함하는지 알아보기
def aa(x, y):
    if x in y:
        return 1
    else:
        return 0

df_test['PREF_in'] = df_test.apply(lambda df2: aa(df2['user_pref'], df2['coupon_pref']), axis=1)

In [91]:
prefecture_location = pd.read_csv('../coupon_data_project2/prefecture_locations_translated_en.csv')

prefecture_location.drop('PREFECTUAL_OFFICE', axis=1, inplace=True)

# user_pref의 위도 경도 가져오기
prefecture_location.rename(columns={'PREF_NAME':'user_pref'},inplace=True)
df_test = pd.merge(df_test, prefecture_location, how='left', on='user_pref')

# spot_pref의 위도 경도 가져오기
prefecture_location.rename(columns={'user_pref':'spot_pref'},inplace=True)
df_test = pd.merge(df_test, prefecture_location, how='left', on='spot_pref')

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [76]:
df_test = df_test.drop_duplicates(['COUPON_ID_hash','USER_ID_hash'], keep='last')

In [92]:
df_test['distance'] = df_test.apply(lambda df_loc : distance(df_loc['LATITUDE_x'] , df_loc['LONGITUDE_x'] ,df_loc['LATITUDE_y'], df_loc['LONGITUDE_y']), axis=1)

In [99]:
test_csv = df_test[['COUPON_ID_hash', 'USER_ID_hash','PREF_in','distance']]

In [100]:
test_csv.to_csv('test_location.csv', encoding='utf-8')